In [0]:
!pip install -U docling

In [0]:
import os
from pathlib import Path
import pandas as pd
from docling.document_converter import DocumentConverter, PdfFormatOption


In [0]:
root_path = Path.cwd()

parent_path = root_path.parent.parent
data_path = parent_path / "out" / "Nano Family" / "docling_chunks.jsonl"
print("Data Path:", data_path)

In [0]:
from pathlib import Path
import sys

def add_project_root(levels_up: int = 1):
    """
    Fügt den Projekt-Root (eine oder mehrere Ebenen über dem aktuellen Notebook)
    zum Python-Suchpfad (sys.path) hinzu.

    Standardmäßig geht es 1 Ebene nach oben:
    z. B. von 'chunking/experiments/' → 'chunking/'
    """
    root = Path.cwd()
    for _ in range(levels_up):
        root = root.parent  # geht jeweils eine Ebene nach oben
    if str(root) not in sys.path:
        sys.path.append(str(root))
        print(f"✅ Added project root to sys.path: {root}")
    else:
        print(f"ℹ️ Project root already in sys.path: {root}")

In [0]:
add_project_root(1)

In [0]:
doc_root = Path("/Workspace/Users/nasiba.tuychieva@gea.com/master_thesis-rag/main/chunking/experiments/Portenta Familiy/documents")
out_dir  = Path("/Workspace/Users/nasiba.tuychieva@gea.com/master_thesis-rag/main/chunking/experiments/Portenta Familiy/out")
print(doc_root)
print(out_dir)

In [0]:
# pdf_path = Path("/Workspace/Users/nasiba.tuychieva@gea.com/master_thesis-rag/main/chunking/experiments/Portenta Familiy/documents/Portenta Family/Portenta C33/Espressif ESP32-C3-MINI-1U.pdf")
# category = pdf_path.parent.parent.name
# product  = pdf_path.parent.name
# filename = pdf_path.stem  # Dateiname ohne .pdf
# parts = filename.split("_")  # ["Elements", "Bluetooth", "Espressif", "ESP32-C3-MINI-1U"]

# element = None
# if parts and parts[0] == "Elements":
#     # nehme die ersten zwei Wörter: Elements_Bluetooth
#     element = "_".join(parts[:2])

# print(element)

In [0]:
import importlib
import process_document

importlib.reload(process_document)

In [0]:
process_document.iterate_product_docs(doc_root = doc_root,out_dir = out_dir) 

In [0]:
%pip install playwright
!playwright install chromium

In [0]:
out_html = "/Workspace/Users/nasiba.tuychieva@gea.com/master_thesis-rag/main/chunking/experiments/Portenta Familiy/out"
p = "/Workspace/Users/nasiba.tuychieva@gea.com/master_thesis-rag/main/chunking/experiments/Portenta Familiy/documents"
from pathlib import Path
print(p)
print(out_html)

In [0]:
%run ../process_document_html.py

In [0]:
from process_document_html import iterate_product_docs

In [0]:
pdf_path =Path(p)
category = pdf_path.parent.parent.name
product  = pdf_path.parent.name
filename = pdf_path.stem  # Dateiname ohne .pdf
parts = filename.split("_")  # ["Elements", "Bluetooth", "Espressif", "ESP32-C3-MINI-1U"]
element = None
if parts and parts[0] == "Elements":
    element = "_".join(parts[:2])
tutorial = None
if parts and parts[0] == "Tutorial":
    tutorial = "_".join(parts[:2])
print(filename)

In [0]:
iterate_product_docs(doc_root=Path(p), out_dir= Path(out_html))

In [0]:
path = "file:/Workspace/Users/nasiba.tuychieva@gea.com/master_thesis-rag/main/chunking/experiments/Portenta Familiy/out/Portenta Family/docling_chunks.jsonl"
df = spark.read.json(path)
from pyspark.sql.functions import col, lit
df.filter(col("chunk_id").contains("Tutorial")).display()

In [0]:
# %pip install playwright
# !playwright install chromium
from bs4 import BeautifulSoup
from pathlib import Path

p = Path(p)
html = p.read_text(encoding="utf-8", errors="ignore")
soup = BeautifulSoup(html, "html.parser")

# Skripte & Styles entfernen
for tag in soup(["script", "style", "noscript"]):
    tag.decompose()

text = soup.get_text(separator="\n")
clean = "\n".join(line.strip() for line in text.splitlines() if line.strip())

Path(out_html).write_text(clean, encoding="utf-8")
print("✅ cleaned static HTML written")




In [0]:
from docling.document_converter import DocumentConverter
doc = DocumentConverter().convert(out_html).document
print(doc.export_to_markdown()[:1000])


In [0]:
%pip install readability-lxml lxml
from readability import Document
from bs4 import BeautifulSoup

html = Path(p).read_text(encoding="utf-8", errors="ignore")
main_html = Document(html).summary()           # extrahiert den “Artikelkörper”
soup = BeautifulSoup(main_html, "lxml")

# Skripte/Styles weg
for tag in soup(["script","style","noscript"]): tag.decompose()

# <li> zu Sätzen bündeln
for ul in soup.find_all(["ul","ol"]):
    items = [li.get_text(" ", strip=True) for li in ul.find_all("li")]
    ul.replace_with( soup.new_tag("p", **{}) )
    ul.string = ". ".join(items) + "."

text = soup.get_text("\n", strip=True)
# Newlines normalisieren
import re
text = re.sub(r"\n{3,}", "\n\n", text)
print(text[:5000])


In [0]:
from bs4 import BeautifulSoup
import re
from pathlib import Path

html = Path(p).read_text(encoding="utf-8", errors="ignore")
soup = BeautifulSoup(html, "lxml")

# Noise raus
for tag in soup(["script","style","noscript","header","footer","nav","aside","form","svg"]):
    tag.decompose()

# Linebreaks vereinheitlichen
for br in soup.find_all("br"):
    br.replace_with("\n")

# Listen in Fließtext umwandeln
for ul in soup.find_all(["ul","ol"]):
    items = [li.get_text(" ", strip=True) for li in ul.find_all("li", recursive=False)]
    para = soup.new_tag("p")
    para.string = ". ".join(i for i in items if i) + "."
    ul.replace_with(para)

# Tabellen in Markdown-Zeilen (grob)
def table_to_md(tbl):
    rows = []
    for tr in tbl.find_all("tr"):
        cells = [c.get_text(" ", strip=True) for c in tr.find_all(["th","td"])]
        if cells: rows.append("| " + " | ".join(cells) + " |")
    return "\n".join(rows)

for tbl in soup.find_all("table"):
    md = table_to_md(tbl)
    tbl.replace_with(soup.new_string(md))

# Fließtext holen & glätten
text = soup.get_text("\n", strip=True)
text = re.sub(r"[ \t]+", " ", text)
text = re.sub(r"\n{3,}", "\n\n", text)

print(text[:50000])


In [0]:
%pip install lxml

In [0]:
%restart_python

In [0]:

from bs4 import BeautifulSoup
import re
from pathlib import Path

html = Path(p).read_text(encoding="utf-8", errors="ignore")
soup = BeautifulSoup(html, "lxml")

# Entferne unnötige Elemente
for tag in soup(["script", "style", "noscript", "header", "footer", "nav", "aside", "form", "svg"]):
    tag.decompose()

# Konvertiere Codeblöcke in Markdown
for pre in soup.find_all("pre"):
    code = pre.get_text("\n", strip=True)
    pre.replace_with("\n```\n" + code + "\n```\n")

# Inline-Code (z. B. <code>setup()</code>)
for c in soup.find_all("code"):
    txt = c.get_text(" ", strip=True)
    c.replace_with(f"`{txt}`")

# Danach alles andere holen
text = soup.get_text("\n", strip=True)
text = re.sub(r"\n{3,}", "\n\n", text)

print(text[:1500])


In [0]:
%pip install docling

In [0]:
from pathlib import Path
from tempfile import NamedTemporaryFile
from bs4 import BeautifulSoup
import re

# 1) HTML laden & bereinigen (deine Logik)
raw_html = Path(p).read_text(encoding="utf-8", errors="ignore")
soup = BeautifulSoup(raw_html, "lxml")

# Noise entfernen
for tag in soup(["script","style","noscript","header","footer","nav","aside","form","svg"]):
    tag.decompose()

# Codeblöcke in Markdown-Fences (bleiben als <pre> Text im HTML stehen)
for pre in soup.find_all("pre"):
    code = pre.get_text("\n", strip=True)
    pre.string = "\n```\n" + code + "\n```\n"   # Inhalt ersetzen, Tag bleibt

# Inline-Code zu `code`
for c in soup.find_all("code"):
    txt = c.get_text(" ", strip=True)
    c.string = f"`{txt}`"

# Optional: überflüssige Leerzeilen in P/LI normalisieren
for el in soup.find_all(["p","li"]):
    if el.string:
        el.string.replace_with(re.sub(r"\s+\n\s+", "\n", el.string))

clean_html = str(soup)  # <- WICHTIG: Struktur beibehalten!

# 2) Docling-Dokument aus dem bereinigten HTML bauen

with NamedTemporaryFile("w", suffix=".html", delete=False, encoding="utf-8") as tmp:
    tmp.write(clean_html)
    tmp_path = tmp.name
from docling.document_converter import DocumentConverter
converter = DocumentConverter()
doc = converter.convert(tmp_path).document

# 3) Chunker & Tokenizer wie gehabt wiederverwenden
from docling_core.transforms.chunker.tokenizer.huggingface import HuggingFaceTokenizer
from transformers import AutoTokenizer
from docling.chunking import HybridChunker

def return_tokenizer():
    tok = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
    return HuggingFaceTokenizer(tokenizer=tok, max_tokens=800)

tokenizer = return_tokenizer()
chunker = HybridChunker(tokenizer=tokenizer, merge_peers=True)

chunks = list(chunker.chunk(dl_doc=doc))

# Beispiel: ersten Kontext-Chunk anzeigen
print("chunks:", len(chunks))
if chunks:
    print(chunker.contextualize(chunk=chunks[0])[:800])


In [0]:
!pip install pikepdf
import pikepdf
with pikepdf.open("/Workspace/Users/nasiba.tuychieva@gea.com/master_thesis-rag/main/chunking/experiments/Portenta Familiy/documents/Portenta Family/Portenta C33/Espressif ESP32-C3-MINI-1U Wi-Fi and Bluetooth.pdf") as pdf:
    pdf.save("/Workspace/Users/nasiba.tuychieva@gea.com/master_thesis-rag/main/chunking/experiments/Portenta Familiy/documents/Portenta Family/Portenta C33/Espressif ESP32-C3-MINI-1U Wi-Fi and Bluetooth_repaired.pdf")


In [0]:
data_path = "file:/Workspace/Users/nasiba.tuychieva@gea.com/master_thesis-rag/main/chunking/experiments/Portenta Familiy/out/Portenta Family/docling_chunks.jsonl"
df = spark.read.json(data_path)
df.display()

In [0]:
df.info()